# Searching for a similar chain in PDB using Biopython and multialign

Для выполнения задачи выравнивания целевого участка из вашего PDB-файла (`6gl.pdb`, цепочка `A`, последовательность `KFNHEAEDLFY`) с аналогичными участками в других PDB-файлах (штаммах) можно использовать **swalign**.  

1. **Извлечение последовательности из `6lzg.pdb`** (целевой участок `KFNHEAEDLFY` в цепи `A`).  
2. **Для каждого штамма (PDB-файла):**  
   - Загружается его последовательность.  
   - Выполняется **выравнивание**. 
   - Находятся **координаты** участка, наиболее похожего на `KFNHEAEDLFY`.  

In [1]:
# !pip install biopython
# !pip install swalign

In [2]:
from glob import glob
import os
from Bio import SeqIO, Align
from Bio.PDB import PDBParser
from Bio.Data.IUPACData import protein_letters_3to1
from Bio.Align import substitution_matrices
import pandas as pd
import swalign

In [3]:
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}


def shorten(x):
    if len(x) % 3 != 0: 
        raise ValueError('Input length should be a multiple of three')

    y = ''
    for i in range(len(x) // 3):
        y += d[x[3 * i : 3 * i + 3]]
    return y

In [4]:
rdb_dir_path = "/home/jovyan/protein-protein-docking/data/RBD"
base_structure_path = "/home/jovyan/protein-protein-docking/data/6lzg.pdb"
target_sequence = "KFNHEAEDLFY" # converted from LYSPHEASNHISGLUALAGLUASPLEUPHETYR

In [5]:
parser = PDBParser()
# aligner = Align.PairwiseAligner()
# aligner.mode = 'local' #   global
# aligner.open_gap_score = -1 # штраф за открытие gap
# aligner.extend_gap_score = -2  # штраф за продолжение gap
# # aligner.match_score = 1.0
# aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")

match = 4
mismatch = 2
scoring = swalign.NucleotideScoringMatrix(match, mismatch)
# scoring = swalign.NucleotideScoringMatrix()

sw = swalign.LocalAlignment(scoring)

In [6]:
results = pd.DataFrame(columns=[
    "PDB_File",
    "Chain", 
    "Start_in_Query", 
    "End_in_Query", 
    "Alignment_Score", 
])

for rdb_pdb_path in glob(os.path.join(rdb_dir_path, "*.pdb")):
    structure = parser.get_structure("A", rdb_pdb_path)
    for model in structure:
        for chain in model:
            chain_sequence = "".join([residue.get_resname() for residue in chain])
            alignment = sw.align(target_sequence, shorten(chain_sequence))
            end_in_query = alignment.q_pos + alignment.r_end - alignment.r_pos
            print(f"Файл {os.path.basename(rdb_pdb_path)}: query pos {alignment.q_pos} - {end_in_query}")
            alignment.dump()
            print("-" *15, "\n")
            results.loc[len(results)] = {
                "PDB_File": os.path.basename(rdb_pdb_path),
                "Chain": chain.id,
                "Start_in_Query": alignment.q_pos,
                "End_in_Query": end_in_query,
                "Alignment_Score": alignment.identity
            }
            
            # Выравнивание с целевой последовательностью
            # alignments = aligner.align(target_sequence, shorten(chain_sequence))
            # best_alignment = alignments[0]  # берем лучшее выравнивание
            
            # # Находим координаты выровненного участка
            # start_in_target = best_alignment.aligned[0][0][0]  # Старт в target
            # start_in_query = best_alignment.aligned[1][0][0]   # Старт в query
            # end_in_target = best_alignment.aligned[0][-1][-1]  # Старт в target
            # end_in_query = best_alignment.aligned[1][-1][-1]   # Старт в query
            
            # print(f"Файл {os.path.basename(rdb_pdb_path)}: совпадение с позиции {start_in_query} до {end_in_query}")
            # print(best_alignment.score, best_alignment.length)
            # print(best_alignment)

            # results.loc[len(results)] = {
            #     "PDB_File": os.path.basename(rdb_pdb_path),
            #     "Chain": chain.id,
            #     "Start_in_Query": start_in_query,
            #     "End_in_Query": end_in_query,
            #     "Alignment_Score": best_alignment.score
            # }

results.to_csv("alignment_results.csv", index=False)

Файл RBD_pdb7XBGmutant_unrelaxed_rank_005_alphafold2_ptm_model_1_seed.pdb: query pos 179 - 190
Query: 180 KPCSVEGPDCYY 191
           | ...|..|..|
Ref  :   1 K-FNHEAEDLFY 11

Score: 29
Matches: 4 (33.3%)
Mismatches: 8
CIGAR: 1M1I10M
--------------- 

Файл RBD_pdb7F5H_unrelaxed_rank_005_alphafold2_ptm_model_1_seed_000.pdb: query pos 202 - 213
Query: 203 KSTGTLEVLFQ 213
           |.....|.||.
Ref  :   1 KFNHEAEDLFY 11

Score: 30
Matches: 4 (36.4%)
Mismatches: 7
CIGAR: 11M
--------------- 

Файл RBD_UVF39818_unrelaxed_rank_003_alphafold2_ptm_model_4_seed_000.pdb: query pos 180 - 191
Query: 181 GFNCYFPLQSY 191
           .||.......|
Ref  :   1 KFNHEAEDLFY 11

Score: 28
Matches: 3 (27.3%)
Mismatches: 8
CIGAR: 11M
--------------- 

Файл RBD_RSB_7DQA_unrelaxed_rank_005_alphafold2_ptm_model_1_seed_000.pdb: query pos 152 - 163
Query: 153 GFNCYFPLQSY 163
           .||.......|
Ref  :   1 KFNHEAEDLFY 11

Score: 28
Matches: 3 (27.3%)
Mismatches: 8
CIGAR: 11M
--------------- 

Файл RBD_pdb7XBGmutan

In [7]:
results

,PDB_File,Chain,Start_in_Query,End_in_Query,Alignment_Score
0,RBD_pdb7XBGmutant_unrelaxed_rank_005_alphafold...,A,179,190,0.333333
1,RBD_pdb7F5H_unrelaxed_rank_005_alphafold2_ptm_...,A,202,213,0.363636
2,RBD_UVF39818_unrelaxed_rank_003_alphafold2_ptm...,A,180,191,0.272727
3,RBD_RSB_7DQA_unrelaxed_rank_005_alphafold2_ptm...,A,152,163,0.272727
4,RBD_pdb7XBGmutant_unrelaxed_rank_003_alphafold...,A,179,190,0.333333
5,RBD_RSB_7WBL_unrelaxed_rank_004_alphafold2_ptm...,A,145,156,0.333333
6,RBD_pdb7XBGmutant_unrelaxed_rank_001_alphafold...,A,179,190,0.333333
7,RBD_rbdKFU_unrelaxed_rank_001_alphafold2_ptm_m...,A,208,219,0.272727
8,RBD_RSB_6M0J_unrelaxed_rank_001_alphafold2_ptm...,A,166,177,0.272727
9,RBD_RSB_7WBL_unrelaxed_rank_001_alphafold2_ptm...,A,145,156,0.333333
